In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip install wandb

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torch.utils.data as data
from tqdm import tqdm


In [ ]:
import wandb
wandb.login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
%cd 'drive/MyDrive/transfer_learning'

/content/drive/MyDrive/transfer_learning


In [6]:
sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }
sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        }
    }
sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'value': 30}
    })

import math

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.001,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(2),
        'max': math.log(4),
      }
    })

In [7]:
sweep_config

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform',
   'max': 1.3862943611198906,
   'min': 0.6931471805599453,
   'q': 1},
  'epochs': {'value': 30},
  'learning_rate': {'distribution': 'uniform', 'max': 0.1, 'min': 0.001},
  'optimizer': {'values': ['adam', 'sgd']}}}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

In [9]:
loss_function = nn.CrossEntropyLoss()

In [11]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        train_data_loader, val_data_loader = build_dataset(config.batch_size)
        network = build_network()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
        for epoch in range(config.epochs):
          print('epoch:',epoch)
          avg_loss, avg_acc = train_epoch(network, train_data_loader, optimizer)
          wandb.log({"loss_train": avg_loss, "accuracy_train": avg_acc, "epoch": epoch})

          avg_loss_eval, avg_acc_eval = test_epoch(network, val_data_loader)    
          wandb.log({"loss_eval": avg_loss_eval, "accuracy_eval": avg_acc_eval})      

In [12]:
def calc_acc(preds, labels):
    preds_max = torch.argmax(preds, 1)
    acc = torch.sum(preds_max == labels.data, dtype=torch.float16) / len(preds)
    return acc

In [13]:
def build_dataset(batch_size):
    
  transform =transforms.Compose([
                                  transforms.Resize((64, 64)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
  ])

  dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/MNIST_persian/MNIST_persian', transform=transform)
  train_size = int(len(dataset)*0.8)
  val_size = len(dataset)-train_size
  train_data ,val_data = data.random_split(dataset,[train_size,val_size])
  train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True,num_workers=1)
  val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False,num_workers=1)
  return train_data_loader, val_data_loader


def build_network():
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = torchvision.models.resnet50(pretrained=True)
  return model.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, train_data_loader, optimizer):
  network.train(True)
  train_loss=0.0
  train_acc=0.0
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  for images,labels in tqdm(train_data_loader):
    images=images.to(device)
    labels=labels.to(device)
    optimizer.zero_grad()
    
    preds_train = network(images)

    loss_train=loss_function(preds_train,labels) # loss_train
    loss_train.backward()

    optimizer.step()

    train_loss += loss_train
    train_acc += calc_acc(preds_train,labels)
    
  total_loss = train_loss/len(train_data_loader)
  total_acc = train_acc/len(train_data_loader)
  print(f"loss_train:{total_loss},accuracy_train:{total_acc}")
  
  return total_loss, total_acc

def test_epoch(network, val_data_loader):
  network.eval()
  test_loss=0.0
  test_acc=0.0
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  for images,labels in tqdm(val_data_loader):    
    images=images.to(device)
    labels=labels.to(device)
    
    preds_test = network(images)

    loss_test=loss_function(preds_test,labels) 

    test_loss += loss_test
    test_acc += calc_acc(preds_test,labels)

  total_loss = test_loss/len(val_data_loader)
  total_acc = test_acc/len(val_data_loader)

  print(f"loss_eval:{total_loss},accuracy_eval:{total_acc}")

  return total_loss, total_acc


In [14]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: qmhx0o3h with config:
wandb: 	batch_size: 3
wandb: 	epochs: 30
wandb: 	learning_rate: 0.043075675165062874
wandb: 	optimizer: adam


epoch: 0


100%|██████████| 320/320 [00:23<00:00, 13.40it/s]


loss_train:3.754723310470581,accuracy_train:0.09942626953125


100%|██████████| 80/80 [00:02<00:00, 35.76it/s]


...

100%|██████████| 240/240 [00:15<00:00, 15.45it/s]


loss_train:0.008373342454433441,accuracy_train:0.9970703125


100%|██████████| 60/60 [00:01<00:00, 33.56it/s]


loss_eval:0.21060937643051147,accuracy_eval:0.94580078125


accuracy_eval,▁▃▄▆▅▇▇▇▆▇▇█▇███▇█████▇███████
accuracy_train,▁▂▃▅▆▆▇▇▇▇▇▇▇▇████████████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss_eval,█▃▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_train,█▄▃▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy_eval,0.9458
accuracy_train,0.99707
epoch,29
loss_eval,0.21061
loss_train,0.00837


In [20]:
model = build_network()
torch.save(model.state_dict(), "/content/drive/MyDrive/transfer_learning/model_mnist.pth")

In [ ]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform',
                               'max': 5.545177444479562,
                               'min': 3.4657359027997265,
                               'q': 1},
                'epochs': {'values': 3},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [ ]:
wandb.init(project="transferlearning_persion_mnist")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.resnet50(pretrained=True)

In [ ]:
# Transfer Learning
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)

In [ ]:
model=model.to(device)

In [ ]:
model

In [ ]:
# freeze
# ct = 0
# for child in model.children():
#   ct += 1
#   if ct < 6:
#     for param in child.parameters():
#         param.requires_grad = False

In [ ]:
config = wandb.config
config.learning_rate = 0.01
config.batch_size = 1
config.epochs = 15

In [ ]:
transform =transforms.Compose([
                                transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
])

dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/MNIST_persian/MNIST_persian', transform=transform)
torch.manual_seed(0)
train_size = int(len(dataset)*0.8)
val_size = len(dataset)-train_size
train_data ,val_data = data.random_split(dataset,[train_size,val_size])

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=config.batch_size, shuffle=True,num_workers=32)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=config.batch_size, shuffle=False,num_workers=32)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# compile
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
loss_function = nn.CrossEntropyLoss()

In [ ]:
def calc_acc(preds, labels):
    preds_max = torch.argmax(preds, 1)
    acc = torch.sum(preds_max == labels.data, dtype=torch.float64) / len(preds)
    return acc

In [ ]:
def train(model, train_data_loader,epoch):
  wandb.watch(model)
  model.train(True)
  train_loss=0.0
  train_acc=0.0
  for images,labels in tqdm(train_data_loader):
    images=images.to(device)
    labels=labels.to(device)
    optimizer.zero_grad()
    
    preds_train = model(images)

    loss_train=loss_function(preds_train,labels) # loss_train
    loss_train.backward()

    optimizer.step()

    train_loss += loss_train
    train_acc += calc_acc(preds_train,labels)
    
  total_loss = train_loss/len(train_data_loader)
  total_acc = train_acc/len(train_data_loader)

  if epoch % 2 == 0:
    wandb.log({"epoch": epoch})
    wandb.log({"loss_train": total_loss})
    wandb.log({"acc_train": total_acc})

  print(f"loss_train:{total_loss},accuracy_train:{total_acc}")

In [ ]:
def test(model, val_data_loader,epoch):
  model.eval()
  test_loss=0.0
  test_acc=0.0
  for images,labels in tqdm(val_data_loader):    
    images=images.to(device)
    labels=labels.to(device)
    
    preds_test = model(images)

    loss_test=loss_function(preds_test,labels) 

    test_loss += loss_test
    test_acc += calc_acc(preds_test,labels)

  total_loss = test_loss/len(val_data_loader)
  total_acc = test_acc/len(val_data_loader)

  if epoch % 2 == 0:
        wandb.log({"loss_eval": total_loss})
        wandb.log({"acc_eval": total_acc})

  print(f"loss_eval:{total_loss},accuracy_eval:{total_acc}")

In [ ]:
for epoch in range(config.epochs):
  print(f'Epoch:{epoch}')
  train(model, train_data_loader, epoch)
  test(model, val_data_loader, epoch)

In [23]:
transform =transforms.Compose([
                                  transforms.Resize((64, 64)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
  ])
import cv2
import numpy as np
model = build_network()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/transfer_learning/model_mnist.pth' ,map_location=torch.device(device)))

model.eval()
img = cv2.imread('/content/drive/MyDrive/MNIST_persian/MNIST_persian/2/2_13.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img = cv2.resize(img, (64, 64))

from PIL import Image
PIL_image = Image.fromarray(img)

tensor = transform(PIL_image).unsqueeze(0).to(device)

# process
pred = model(tensor)

# postprocess
pred = pred.cpu().detach().numpy()
pred = np.argmax(pred)

print('prediction:', pred)

prediction: 549
